# Hindi to English translation

Primary python code to clean and invoke the models.

In [14]:
#!pip install inltk
#!pip install nest_asyncio
#!pip install torch==1.3.1

In [15]:
import nest_asyncio
nest_asyncio.apply()

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import string
import re
from pickle import dump
from unicodedata import normalize
import codecs

import sys
from collections import Counter

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from inltk.inltk import setup
from inltk.inltk import tokenize
from inltk.inltk import get_embedding_vectors


#from indicnlp import common
#from indicnlp import loader
#from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

In [20]:
setup('hi')
setup('pa')
setup('bn')

Done!


RuntimeError: Cannot close a running event loop

In [ ]:
hindi_trainfile = "./dakshina_dataset_v1.0/hi/romanized/hi.romanized.rejoined.dev.native.txt"
hindi_trainYFile = "./dakshina_dataset_v1.0/hi/romanized/hi.romanized.rejoined.dev.roman.txt"
hindi_testfile = "./dakshina_dataset_v1.0/hi/romanized/hi.romanized.rejoined.test.native.txt"
hindi_testYfile = "./dakshina_dataset_v1.0/hi/romanized/hi.romanized.rejoined.test.roman.txt"
hindi_validationAndYfile = "./dakshina_dataset_v1.0/hi/romanized/hi.romanized.rejoined.tsv"


bengali_trainfile = "./dakshina_dataset_v1.0/bn/romanized/bn.romanized.rejoined.dev.native.txt"
bengali_trainYfile = "./dakshina_dataset_v1.0/bn/romanized/bn.romanized.rejoined.dev.roman.txt"
bengali_testfile = "./dakshina_dataset_v1.0/bn/romanized/bn.romanized.rejoined.test.native.txt"
bengali_testYfile = "./dakshina_dataset_v1.0/bn/romanized/bn.romanized.rejoined.test.roman.txt"
bengali_validationAndYfile = "./dakshina_dataset_v1.0/bn/romanized/bn.romanized.rejoined.tsv"

punjabi_trainfile = "./dakshina_dataset_v1.0/pa/romanized/pa.romanized.rejoined.dev.native.txt"
punjabi_trainYfile = "./dakshina_dataset_v1.0/pa/romanized/pa.romanized.rejoined.dev.roman.txt"
punjabi_testfile = "./dakshina_dataset_v1.0/pa/romanized/pa.romanized.rejoined.test.native.txt"
punjabi_testYfile = "./dakshina_dataset_v1.0/pa/romanized/pa.romanized.rejoined.test.roman.txt"
punjabi_validationAndYfile = "./dakshina_dataset_v1.0/pa/romanized/pa.romanized.rejoined.tsv"

In [ ]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [ ]:
hindiOrigTr_ls = load_doc(hindi_trainfile).splitlines()
hindiOrigTrY_ls = load_doc(hindi_trainYFile).splitlines()
hindiOrigTe_ls = load_doc(hindi_testfile).splitlines()
hindiOrigTeY_ls = load_doc(hindi_testYfile).splitlines()
hindiOrigValAndY_ls = load_doc(hindi_validationAndYfile).splitlines()

bengaliOrigTr_ls = load_doc(bengali_trainfile).splitlines()
bengaliOrigTrY_ls = load_doc(bengali_trainYfile).splitlines()
bengaliOrigTe_ls = load_doc(bengali_testfile).splitlines()
bengaliOrigTeY_ls = load_doc(bengali_testYfile).splitlines()
bengaliOrigValAndY_ls = load_doc(bengali_validationAndYfile).splitlines()

punjabiOrigTr_ls = load_doc(punjabi_trainfile).splitlines()
punjabiOrigTrY_ls = load_doc(punjabi_trainYfile).splitlines()
punjabiOrigTe_ls = load_doc(punjabi_testfile).splitlines()
punjabiOrigTeY_ls = load_doc(punjabi_testYfile).splitlines()
punjabiOrigValAndY_ls = load_doc(punjabi_validationAndYfile).splitlines()

In [ ]:
hindiOrigVal_ls = [val.split("\t")[0] for val in hindiOrigValAndY_ls ]
hindiOrigValY_ls = [val.split("\t")[1] for val in hindiOrigValAndY_ls ]

bengaliOrigVal_ls = [val.split("\t")[0] for val in bengaliOrigValAndY_ls ]
bengaliOrigValY_ls = [val.split("\t")[1] for val in bengaliOrigValAndY_ls ]

punjabiOrigVal_ls = [val.split("\t")[0] for val in punjabiOrigValAndY_ls ]
punjabiOrigValY_ls = [val.split("\t")[1] for val in punjabiOrigValAndY_ls ]

## Cleaning up data
1. Remove punctuations from sentence
2. Removing non-printable characters.
3. Removing words that contain non-alphabetic characters.

In [ ]:
def unicodeReplace(line):
    invalidUnicode = ['\u200c', '.', '।']
    for val in invalidUnicode:
        line = line.replace(val, '')
    return line
    
def clean_lines(lines):
    cleaned = []
    # prepare regex for char filtering
    #re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for line in lines:
        #remove unwanted unicode characters:
        line = unicodeReplace(line)
        #print(line)
        # normalize unicode characters
        #line = normalize('NFD', line).encode('ascii', 'ignore')
        #line = line.decode('UTF-8')
        # tokenize on white space
        line = line.split()
        # convert to lower case
        line = [word.lower() for word in line]
        # remove punctuation from each token
        line = [word.translate(table) for word in line]
        # remove non-printable chars form each token
        #line = [re_print.sub('', w) for w in line]
        # remove tokens with numbers in them
        #line = [word for word in line if word.isalpha()]
        # store as string
        cleaned.append(' '.join(line))
    return cleaned

In [ ]:
hindi_trainClfile = clean_lines(hindiOrigTr_ls)
hindi_trainYClfile= clean_lines(hindiOrigTrY_ls)
hindi_testClfile= clean_lines(hindiOrigTe_ls)
hindi_testYClfile= clean_lines(hindiOrigTeY_ls)
hindi_valClfile= clean_lines(hindiOrigVal_ls)
hindi_valYClfile= clean_lines(hindiOrigValY_ls)


bengali_trainClfile= clean_lines(bengaliOrigTr_ls)
bengali_trainYClfile= clean_lines(bengaliOrigTrY_ls)
bengali_testClfile= clean_lines(bengaliOrigTe_ls)
bengali_testYClfile= clean_lines(bengaliOrigTeY_ls)
bengali_valClfile= clean_lines(bengaliOrigVal_ls)
bengali_valYClfile=clean_lines(bengaliOrigValY_ls)


punjabi_trainClfile= clean_lines(punjabiOrigTr_ls)
punjabi_trainYClfile= clean_lines(punjabiOrigTrY_ls)
punjabi_testClfile= clean_lines(punjabiOrigTe_ls)
punjabi_testYClfile= clean_lines(punjabiOrigTeY_ls)
punjabi_valClfile= clean_lines(punjabiOrigVal_ls)
punjabi_valYClfile= clean_lines(punjabiOrigValY_ls)

In [ ]:
print(hindiOrigTrY_ls[0:5])
print(hindi_trainYClfile[0:5])

In [ ]:
print(hindiOrigTr_ls[0:5])
print(hindi_trainClfile[0:5])

In [ ]:
print(punjabiOrigTr_ls[0:5])
print(punjabi_trainClfile[0:5])
print(punjabi_trainClfile[0][-1])

## Tokenize the indic words using the simplest method

In [ ]:
# create a frequency table for all words
def to_vocab(lines):
    vocab = Counter()
    for line in lines:
        tokens = line.split()
        vocab.update(tokens)
    return vocab
# remove all words with a frequency below a threshold
def trim_vocab(vocab, min_occurance):
    tokens = [k for k,c in vocab.items() if c >= min_occurance]
    return set(tokens)
# mark all OOV with "unk" for all lines
def update_dataset(lines, vocab):
    new_lines = list()
    for line in lines:
        new_tokens = list()
        for token in line.split():
            if token in vocab:
                new_tokens.append(token)
            else:
                new_tokens.append('unk')
        new_line = ' '.join(new_tokens)
        new_lines.append(new_line)
    return new_lines
# save a list of clean sentences to file
def save_clean_sentences(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)
    

def tokenize(x, tokenizer):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x)

def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')

In [21]:
#inltk tokenizer
example_sent = "बहुत समय से मिले नहीं"
# Tokenize the sentence
example_sent_tokens = tokenize(example_sent,'hi')
print(example_sent_tokens)
# Get the embedding vector for each token
example_sent_vectors = get_embedding_vectors(example_sent, 'hi')

['▁बहुत', '▁समय', '▁से', '▁मिले', '▁नहीं']


ModuleAttributeError: 'LSTM' object has no attribute '_flat_weights_names'

In [ ]:
def cleanAndTrimVocab(listData, fileName, tokenizer):
    listData_count = to_vocab(listData)
    listData_vocab = trim_vocab(listData_count, 5)
    # mark out of vocabulary words
    listData_updated = update_dataset(listData, listData_vocab)
    
    #tokenize data
    tokenized_data = tokenize(listData_updated, tokenizer)
    
    #pad with empty spaces
    listData_updated = pad(tokenized_data)
    
    save_clean_sentences(listData_updated, fileName)

In [ ]:
hindiTokenizer = Tokenizer()
englishTokenizer = Tokenizer()
bengaliTokenizer = Tokenizer()
punjabiTokenizer = Tokenizer()

baseCleanPath = "/Users/mita/Documents/GitHub/MscPrg/Coding/PreparedData/"
cleanAndTrimVocab(hindi_trainClfile, baseCleanPath +"hi/trainclean.pkl", hindiTokenizer)
cleanAndTrimVocab(hindi_trainYClfile , baseCleanPath +"hi/trainYclean.pkl", englishTokenizer)
cleanAndTrimVocab(hindi_testClfile, baseCleanPath +"hi/testclean.pkl", hindiTokenizer) 
cleanAndTrimVocab(hindi_testYClfile, baseCleanPath +"hi/testYclean.pkl", englishTokenizer) 
cleanAndTrimVocab(hindi_valClfile, baseCleanPath +"hi/valclean.pkl", hindiTokenizer) 
cleanAndTrimVocab(hindi_valYClfile, baseCleanPath +"hi/valYclean.pkl", englishTokenizer) 


cleanAndTrimVocab(bengali_trainClfile, baseCleanPath +"bn/trainclean.pkl", bengaliTokenizer) 
cleanAndTrimVocab(bengali_trainYClfile, baseCleanPath +"bn/trainYclean.pkl", englishTokenizer)
cleanAndTrimVocab(bengali_testClfile, baseCleanPath +"bn/testclean.pkl", bengaliTokenizer) 
cleanAndTrimVocab(bengali_testYClfile, baseCleanPath +"bn/testYclean.pkl", englishTokenizer) 
cleanAndTrimVocab(bengali_valClfile, baseCleanPath +"bn/valclean.pkl", bengaliTokenizer) 
cleanAndTrimVocab(bengali_valYClfile, baseCleanPath +"bn/valYclean.pkl", englishTokenizer)


cleanAndTrimVocab(punjabi_trainClfile, baseCleanPath +"pa/trainclean.pkl", punjabiTokenizer) 
cleanAndTrimVocab(punjabi_trainYClfile, baseCleanPath +"pa/trainYclean.pkl", englishTokenizer) 
cleanAndTrimVocab(punjabi_testClfile, baseCleanPath +"pa/testclean.pkl", punjabiTokenizer) 
cleanAndTrimVocab(punjabi_testYClfile, baseCleanPath +"pa/testYclean.pkl", englishTokenizer)
cleanAndTrimVocab(punjabi_valClfile, baseCleanPath +"pa/valclean.pkl", punjabiTokenizer)
cleanAndTrimVocab(punjabi_valYClfile, baseCleanPath +"pa/valYclean.pkl", englishTokenizer)


## Tokenize hindi words using Indic NLP:
One challenge is to convert the text into inputs which can be handled by the different models.
Thus using Indic NLP to convert the text into phoenetic sounds for treatement


In [ ]:
#!pip install indic-nlp-library

In [ ]:
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME="/Users/mita/Documents/GitHub/MscPrg/Coding/indic_nlp_library-master"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES="/Users/mita/Documents/GitHub/MscPrg/Coding/indic_nlp_resources-master"

In [ ]:
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
common.set_resources_path(INDIC_NLP_RESOURCES)

loader.load()

In [ ]:
input_text="\u0958 \u0915\u093c"
remove_nuktas=False
factory=IndicNormalizerFactory()
normalizer=factory.get_normalizer("hi", remove_nuktas=False)
output_text=normalizer.normalize(input_text)

print(input_text)
print()
print(output_text)

print('Before normalization')
print(' '.join([ hex(ord(c)) for c in input_text ] ))
print('Length: {}'.format(len(input_text)))
print()    
print('After normalization')
print(' '.join([ hex(ord(c)) for c in output_text ] ))
print('Length: {}'.format(len(output_text)))   

Done!
